In [1]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import os
import tensorflow as tf
import tensorflow_data_validation as tfdv
import tensorflow_model_analysis as tfma
import tempfile
from tfx.components.evaluator.component import Evaluator
from tfx.components.example_gen.import_example_gen.component import ImportExampleGen
from tfx.components.example_validator.component import ExampleValidator
from tfx.components.model_validator.component import ModelValidator
from tfx.components.pusher.component import Pusher
from tfx.components.schema_gen.component import SchemaGen
from tfx.components.statistics_gen.component import StatisticsGen
from tfx.components.trainer.component import Trainer
from tfx.components.transform.component import Transform
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.proto import example_gen_pb2
from tfx.proto import evaluator_pb2
from tfx.proto import pusher_pb2
from tfx.proto import trainer_pb2
from tfx.utils.dsl_utils import external_input
#from tfx.utils.types import get_split_uri
from tfx.types.artifact_utils import get_split_uri

In [5]:
tf.enable_eager_execution()
tf.executing_eagerly()


AttributeError: module 'tensorflow' has no attribute 'enable_eager_execution'

In [6]:
_root = '.'
_data_root = _root + '/data'
_serving_model_dir = os.path.join(tempfile.mkdtemp(),'serving_model/cifar-10')

In [7]:
context = InteractiveContext()

In [8]:
input_split = example_gen_pb2.Input(splits=[
    example_gen_pb2.Input.Split(name='train', pattern='train.tfrecord'),
    example_gen_pb2.Input.Split(name='eval', pattern='test.tfrecord')
])
examples = external_input(_data_root)
example_gen = ImportExampleGen(input_base=examples, input_config=input_split)

context.run(example_gen)

ExecutionResult(
    component_id: ImportExampleGen
    execution_id: 1
    outputs:
        examples: Channel(
            type_name: Examples
            artifacts: [Artifact(type_name: Examples, uri: /tmp/tfx-interactive-2020-03-19T10_30_32.727775-tanvk6if/ImportExampleGen/examples/1, id: 2)]
        ))

In [13]:
statistics_gen = StatisticsGen(input_data=example_gen.outputs['examples'])
 
context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: Channel(
            type_name: ExampleStatistics
            artifacts: [Artifact(type_name: ExampleStatistics, uri: /tmp/tfx-interactive-2020-03-19T10_30_32.727775-tanvk6if/StatisticsGen/statistics/2, id: 3)]
        ))

In [14]:
artifact_list = statistics_gen.outputs['output'].get()
train_artifact_uri = get_split_uri(artifact_list, 'train')
train_stats_path = os.path.join(train_artifact_uri, 'stats_tfrecord')
eval_artifact_uri = get_split_uri(artifact_list, 'eval')
eval_stats_path = os.path.join(eval_artifact_uri, 'stats_tfrecord')

In [10]:
train_stats = tfdv.load_statistics(train_stats_path)
eval_stats = tfdv.load_statistics(eval_stats_path)

tfdv.visualize_statistics(lhs_statistics=eval_stats, rhs_statistics=train_stats,
                          lhs_name='EVAL_DATASET', rhs_name='TRAIN_DATASET')

W0903 20:22:17.293728 140191945226048 deprecation.py:323] From /opt/interactive_pipeline/venv/lib/python3.6/site-packages/tensorflow_data_validation/utils/stats_gen_lib.py:357: tf_record_iterator (from tensorflow.python.lib.io.tf_record) is deprecated and will be removed in a future version.
Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


In [15]:
infer_schema = SchemaGen(stats=statistics_gen.outputs['output'],
                         infer_feature_shape=True)

context.run(infer_schema)

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: Channel(
            type_name: Schema
            artifacts: [Artifact(type_name: Schema, uri: /tmp/tfx-interactive-2020-03-19T10_30_32.727775-tanvk6if/SchemaGen/schema/3, id: 4)]
        ))

In [16]:
schema_dir = infer_schema.outputs['output'].get()[0].uri
schema_path = os.path.join(schema_dir, 'schema.pbtxt')

schema = tfdv.load_schema_text(schema_path)
tfdv.display_schema(schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'image_raw',BYTES,required,,-
'label',INT,required,,-


In [17]:
validate_stats = ExampleValidator(stats=statistics_gen.outputs['output'],
                                  schema=infer_schema.outputs['output'])

context.run(validate_stats)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 4
    outputs:
        anomalies: Channel(
            type_name: ExampleAnomalies
            artifacts: [Artifact(type_name: ExampleAnomalies, uri: /tmp/tfx-interactive-2020-03-19T10_30_32.727775-tanvk6if/ExampleValidator/anomalies/4, id: 5)]
        ))

In [18]:
validation_dir = validate_stats.outputs['output'].get()[0].uri
anomalies_path = os.path.join(validation_dir, 'anomalies.pbtxt')

anomalies = tfdv.load_anomalies_text(anomalies_path)
tfdv.display_anomalies(anomalies)

In [20]:
_module_file = os.path.join(_root, 'utils.py')

transform = Transform(
    input_data=example_gen.outputs['examples'],
    schema=infer_schema.outputs['output'],
    module_file=_module_file)

context.run(transform)

Instructions for updating:
Schema is a deprecated, use schema_utils.schema_from_feature_spec to create a `Schema`
Instructions for updating:
Use fn_output_signature instead
Instructions for updating:
Use ref() instead.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: /tmp/tfx-interactive-2020-03-19T10_30_32.727775-tanvk6if/Transform/transform_graph/5/.temp_path/tftransform_tmp/220e84be4e7e409cb539f2b2ea209503/saved_model.pb
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


ExecutionResult(
    component_id: Transform
    execution_id: 5
    outputs:
        transform_graph: Channel(
            type_name: TransformGraph
            artifacts: [Artifact(type_name: TransformGraph, uri: /tmp/tfx-interactive-2020-03-19T10_30_32.727775-tanvk6if/Transform/transform_graph/5, id: 6)]
        )
        transformed_examples: Channel(
            type_name: Examples
            artifacts: [Artifact(type_name: Examples, uri: /tmp/tfx-interactive-2020-03-19T10_30_32.727775-tanvk6if/Transform/transformed_examples/5, id: 7)]
        ))

In [21]:
trainer = Trainer(
    module_file=_module_file,
    examples=transform.outputs['transformed_examples'],
    schema=infer_schema.outputs['output'],
    transform_output=transform.outputs['transform_output'],
    train_args=trainer_pb2.TrainArgs(num_steps=100),
    eval_args=trainer_pb2.EvalArgs(num_steps=50))
context.run(trainer)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
image_raw_xf (InputLayer)    [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 30, 30, 64)        1792      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 32)        18464     
_________________________________________________________________
flatten (Flatten)            (None, 25088)             0         
_________________________________________________________________
dense (Dense)                (None, 10)                250890    
Total params: 271,146
Trainable params: 271,146
Non-trainable params: 0
_________________________________________________________________
INFO:tensorflow:Using the Keras model provided.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tfx-interactive-2020-0

ExecutionResult(
    component_id: Trainer
    execution_id: 6
    outputs:
        model: Channel(
            type_name: Model
            artifacts: [Artifact(type_name: Model, uri: /tmp/tfx-interactive-2020-03-19T10_30_32.727775-tanvk6if/Trainer/model/6, id: 8)]
        ))

In [22]:
OVERALL_SLICE_SPEC = evaluator_pb2.SingleSlicingSpec()

ALL_SPECS = [
    OVERALL_SLICE_SPEC,
]

In [23]:
model_analyzer = Evaluator(
    examples=example_gen.outputs['examples'],
    model_exports=trainer.outputs['output'],
    feature_slicing_spec=evaluator_pb2.FeatureSlicingSpec(
        specs=ALL_SPECS
    ))

context.run(model_analyzer)

Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.loader.load or tf.compat.v1.saved_model.load. There will be a new function for importing SavedModels in Tensorflow 2.0.
INFO:tensorflow:Restoring parameters from /tmp/tfx-interactive-2020-03-19T10_30_32.727775-tanvk6if/Trainer/model/6/eval_model_dir/1584581689/variables/variables
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.get_tensor_from_tensor_info or tf.compat.v1.saved_model.get_tensor_from_tensor_info.
INFO:tensorflow:ExampleCount post export metric: could not find any of the standard keys in predictions_dict (keys were: dict_keys(['dense']))
INFO:tensorflow:Using the first key from predictions_dict: dense


ExecutionResult(
    component_id: Evaluator
    execution_id: 7
    outputs:
        evaluation: Channel(
            type_name: ModelEvaluation
            artifacts: [Artifact(type_name: ModelEvaluation, uri: /tmp/tfx-interactive-2020-03-19T10_30_32.727775-tanvk6if/Evaluator/evaluation/7, id: 9)]
        )
        blessing: Channel(
            type_name: ModelBlessing
            artifacts: [Artifact(type_name: ModelBlessing, uri: /tmp/tfx-interactive-2020-03-19T10_30_32.727775-tanvk6if/Evaluator/blessing/7, id: 10)]
        ))

In [19]:
PATH_TO_RESULT = model_analyzer.outputs['output'].get()[0].uri
tfma_result = tfma.load_eval_result(PATH_TO_RESULT)
tfma.view.render_slicing_metrics(tfma_result)

SlicingMetricsViewer(config={'weightedExamplesColumn': 'post_export_metrics/example_count'}, data=[{'slice': '…

In [20]:
model_validator = ModelValidator(
    examples=example_gen.outputs['examples'],
    model=trainer.outputs['output'])

context.run(model_validator)

ExecutionResult(
    component_id: ModelValidator
    execution_id: 8
    outputs:
        blessing: Channel(
            type_name: ModelBlessingPath
            artifacts: [Artifact(type_name: ModelBlessingPath, uri: /tmp/tfx-interactive-2019-09-03T20_21_55.462505-u52yuvqc/ModelValidator/blessing/8/, split: , id: 13)]
        ))

In [ ]:
pusher = Pusher(
    model_export=trainer.outputs['output'],
    model_blessing=model_validator.outputs['blessing'],
    push_destination=pusher_pb2.PushDestination(
        filesystem=pusher_pb2.PushDestination.Filesystem(
            base_directory=_serving_model_dir)))

context.run(pusher)